In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
from functools import wraps
import types


class Profiled:
    def __init__(self, func):
        print('Profiled.__init__')
        wraps(func)(self)
        self.ncalls = 0

    def __call__(self, *args, **kwargs):
        print('Profiled.__call__')
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)

    def __get__(self, obj, objtype):
        print('Profiled.__get__')
        if obj is None:
            return self
        else:
            return types.MethodType(self, obj)


"""
Init

    >>> @Profiled
    ... def add(x, y):
    ...     return x + y
    Profiled.__init__
    >>> class Spam:
    ...     @Profiled
    ...     def bar(self, x):
    ...         print(self, x)
    Profiled.__init__

Inspection

    >>> add(2, 3)
    Profiled.__call__
    5
    >>> add(4, 5)
    Profiled.__call__
    9
    >>> add.ncalls
    2
    >>> s = Spam()
    >>> s.bar(1)  # doctest: +ELLIPSIS
    Profiled.__get__
    Profiled.__call__
    <__main__.Spam object at ...> 1
    >>> s.bar(2)  # doctest: +ELLIPSIS
    Profiled.__get__
    Profiled.__call__
    <__main__.Spam object at ...> 2
    >>> Spam.bar.ncalls
    Profiled.__get__
    2
    >>> Spam.bar  # doctest: +ELLIPSIS
    Profiled.__get__
    <__main__.Profiled object at ...>
    >>> s.bar  # doctest: +ELLIPSIS
    Profiled.__get__
    <bound method Spam.bar of <__main__.Spam object at ...>>

"""

doctest.testmod()

TestResults(failed=0, attempted=11)

In [4]:
import types
from functools import wraps


def profiled(func):
    ncalls = 0

    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal ncalls
        ncalls += 1
        return func(*args, **kwargs)

    wrapper.ncalls = lambda: ncalls
    return wrapper


"""

>>> @profiled
... def add(x, y):
...     return x + y
>>> add(2, 3)
5
>>> add(4, 5)
9
>>> add.ncalls()
2
"""

doctest.testmod()

TestResults(failed=0, attempted=4)